<a href="https://colab.research.google.com/github/kanishkd123/resparser/blob/main/Run1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install docx2txt
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
import PyPDF2
import docx2txt
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string

output = []
resumelist = []
for filename in os.listdir('.'):
    if (filename.endswith('.pdf')) or filename.endswith('.docx'):
        filename = [filename]
        resumelist.append(filename)

def preprocess_resumepdf():
    pdfFileObj = open(name, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pageObj = pdfReader.getPage(0)
    resume = (pageObj.extractText())
    pdfFileObj.close()
    output.append([resume])
    return output

def preprocess_resumeword():
    text = docx2txt.process(name)
    output.append([text])
    return output

for i in range(len(resumelist)):
    name = resumelist[i][0]
    if '.docx' in name:
        preprocess_resumeword()
    else:
        preprocess_resumepdf()

def removestopwords(dataset):
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(str(dataset))
    result = [w for w in word_tokens if w not in stop_words]
    result = [w.lower() for w in result if w.isalpha()]
    return result

def stemming(data):
    stemmer = PorterStemmer()
    tokens = word_tokenize(str(data))
    new_text = ''
    result = []
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
        result.append(new_text)
    return result


jd_name = 'jobs/' + os.listdir('jobs/')[0]
jd = docx2txt.process(jd_name)

stemming(jd)
removestopwords(jd)
removestopwords(output)
stemming(output)

In [ ]:
output

In [ ]:
import logging
from gensim import models
from gensim import corpora
from collections import defaultdict

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

documents = []
for i in range(len(output)):
    resume = str(output[i][0])
    documents.append(resume)

stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
from gensim import similarities

index = similarities.MatrixSimilarity(lsi[corpus])

doc = str(jd)
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]

sims = index[vec_lsi]
x = (list(enumerate(sims)))

sims = sorted(enumerate(sims), key=lambda item: -item[1])
for doc_position, doc_score in sims:
    w = [(doc_score, documents[doc_position])]


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def percentagematch():
    result = []
    for i in range(len(output)):
        resume = output[i][0]
        text = [resume, jd]
        cv = CountVectorizer()
        count_matrix = cv.fit_transform(text)
        match_percentage = cosine_similarity(count_matrix)[0][1] * 100
        match_percentage = round(match_percentage, 2)
        print(match_percentage)
        result.append(match_percentage)
    return result

def output1():
    scores_aux = percentagematch()
    d = {'Name': resumelist, 'Match Score': scores_aux, 'Doc2VecScore': x}
    return d

def getscores(data_aux):
    tfIdfVectorizer = TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(data_aux)
    df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    df = df.sort_values('TF-IDF', ascending=False)
    return df.head(25)

jdscores = [getscores([jd])]
def scores():
    resumescore = []
    for i in range(len(output)):
        resume = output[i][0]
        resume = [resume]
        resumescore.append(getscores(resume))
    return resumescore

data = pd.DataFrame(data=output1())
data = data.sort_values(by='Match Score', ascending=False)
print(data)

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', None, pattern)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text



In [ ]:
import re

def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

In [ ]:
import re

def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [ ]:
import re
import spacy
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII']

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.string.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education


In [ ]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
def extract_experience(resume_text):
    wordnet_lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # word tokenization 
    word_tokens = nltk.word_tokenize(resume_text)

    # remove stop words and lemmatize  
    filtered_sentence = [w for w in word_tokens if not w in stop_words and wordnet_lemmatizer.lemmatize(w) not in stop_words] 
    sent = nltk.pos_tag(filtered_sentence)

    # parse regex
    cp = nltk.RegexpParser('P: {<NNP>+}')
    cs = cp.parse(sent)
    # for i in cs.subtrees(filter=lambda x: x.label() == 'P'):
    #     print(i)
    
    test = []
    
    for vp in list(cs.subtrees(filter=lambda x: x.label()=='P')):
        test.append(" ".join([i[0] for i in vp.leaves() if len(vp.leaves()) >= 2]))

    # Search the word 'experience' in the chunk and then print out the text after it
    x = [x[x.lower().index('experience') + 10:] for i, x in enumerate(test) if x and 'experience' in x.lower()]
    return x

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
#Extract experience 2:
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
stop_words = set(stopwords.words('english'))

def get_first_title(title):
    # keep "co-founder, co-ceo, etc"
    title = re.sub(r"[Cc]o[\-\ ]","", title)
    split_titles = re.split(r"\,|\-|\||\&|\:|\/|and", title)
    return split_titles[0].strip()

def get_title_features(title):
    features = {}
    word_tokens = nltk.word_tokenize(resume_text_data[0])
    filtered_words = [w for w in word_tokens if not w in stop_words] 
    for word in filtered_words:
        features['contains({})'.format(word.lower())] = True
    if len(filtered_words) > 0:
        first_key = 'first({})'.format(filtered_words[0].lower())
        last_key = 'last({})'.format(filtered_words[-1].lower())
        features[first_key] = True
        features[last_key] = True
    return features

# Responsibilities
responsibilities_features = [
    (
         get_title_features(job_title["title"]),
         job_title["responsibility"]
    )
    for job_title in raw_job_titles
    if job_title["responsibility"] is not None
]
# Departments
departments_features = [
    (
         get_title_features(job_title["title"]),
         job_title["department"]
    )
    for job_title in raw_job_titles
    if job_title["department"] is not None
]

# Responsibilities
r_size = int(len(responsibilities_features) * 0.5)
r_train_set = responsibilities_features[r_size:]
r_test_set = responsibilities_features[:r_size]
responsibilities_classifier = nltk.NaiveBayesClassifier.train(
    r_train_set
)
print("Responsibility classification accuracy: {}".format(
    nltk.classify.accuracy(
        responsibilities_classifier,
        r_test_set
    )
))# Departments
d_size = int(len(departments_features) * 0.5)
d_train_set = departments_features[d_size:]
d_test_set = departments_features[:d_size]
departments_classifier = nltk.NaiveBayesClassifier.train(
    d_train_set
)
print("Department classification accuracy: {}".format(
    nltk.classify.accuracy(
        departments_classifier,
        d_test_set
    )
))
title = "Director of Communications"
responsibility = responsibilities_classifier.classify(
    get_title_features(title)
)
department = departments_classifier.classify(
    get_title_features(title)
)
print("Job title: '{}'".format(title))
print("Responsibility: '{}'".format(responsibility))
print("Department: '{}'".format(department))


In [ ]:
raw_job_titles = [
    {
        "title": "Business Development",
        "responsibility": "Strategy",
        "department": "Business Development"
    },
    {
        "title": "Inside Sales Consultant",
        "responsibility": "Execution",
        "department": "Sales"
    }
]

In [ ]:
import re
def extract_date_ranges(format,resume_text):
  t = []
  f = []
  m = re.finditer(format,resume_text.replace('\n', ''))
  for x in m:
    t.append(x.group(0))
  idx = 0
  while idx < len(t):
    try:
      f.append(t[idx] + ' - ' + t[idx+1])
    except:
      break
    idx+=2
  return f

In [ ]:
dateformat1='((January|February|March|April|May|June|July|August|September|October|November|December)\\d{4})|(Current|Present)'
dateformat2='(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\\d{4}'
dateformat3='(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\\d{2}'
dateformat4='\d{4}-\d{4}'

In [ ]:
output

[[' \n \nTAY\n \nYHEE LIN, AMANDA\n \n10 Toh Yi Road, Singapore 596493\n \n\n \n[+6\n5] 98300688 \n\n \namandatayyl@gmail.com\n \n \n \n \n  \n \nFundedHere\n, Singapore \n \n \n \n \n \n \n \n \n \n \n      \n2017\n-\npresent\n \nHead\n, \nInvestor Network\n \n\n \nFundedHere is the first MAS\n-\nlicensed debt\n-\nand\n-\nequity crowdfunding platform; raising funds for companies on the catalyst and startups.\n \n\n \nR\nesponsible \nfor building\n \nout Accredited Investors \nnetwork of ultra/\nhigh \nnet\nworth\n \nindiv\niduals, institutional investors\n \nand family offices.\n \n\n \nManag\ne investment \nmemorandum\n, investor r\neporting\n \nand \nstrategic client meetings.\n \n \nPartners Group AG, Singapore \n \n \n \n \n \n \n \n \n \n          \n2015\n-\n2016\n \nAssociate, \nSenior Product Manager\n, Product Management department\n \n\n \nProduct manager for the flags\nhip direct investments program; \nresponsibilities include providing investor updates on their portfolios,\

In [ ]:
extraction = []
for resume_text_data in output:
    dates=extract_date_ranges(dateformat1,resume_text_data[0])
    if not dates:
      dates = extract_date_ranges(dateformat2,resume_text_data[0])
      if not dates:
        dates = extract_date_ranges(dateformat3,resume_text_data[0])
    extraction.append({
        'Name':extract_name(resume_text_data[0]),
        'Email':extract_email(resume_text_data[0]),
        'Phone':extract_mobile_number(resume_text_data[0]),
        'Experience':extract_experience(resume_text_data[0]),
        'Date_ranges': dates
        })
extraction

[{'Date_ranges': [],
  'Email': 'amandatayyl@gmail.com',
  'Experience': [],
  'Name': 'YHEE LIN',
  'Phone': '9830068'},
 {'Date_ranges': ['Present - May 2013',
   'May 2013 - May 2013',
   'August 2006 - Present'],
  'Email': 'sijie.wee@gmail.com',
  'Experience': [' CITIBANK SINGAPORE Present CITIGOLD TREASURY SALES MANAGER'],
  'Name': 'WEE SI',
  'Phone': '659363063'}]

In [ ]:
df = pd.DataFrame(extraction)
df.to_csv('test.csv')

# Different perspective

In [ ]:
!pip install PyPDF2

In [ ]:
import PyPDF2
reader = PyPDF2.PdfFileReader('ABN_Yee Choon Kit_Relationship Manager.pdf')
reader.documentInfo

In [ ]:
!pip install pdfminer.six

In [ ]:
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def convert_pdf_to_string(file_path):

	output_string = StringIO()
	with open(file_path, 'rb') as in_file:
	    parser = PDFParser(in_file)
	    doc = PDFDocument(parser)
	    rsrcmgr = PDFResourceManager()
	    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
	    interpreter = PDFPageInterpreter(rsrcmgr, device)
	    for page in PDFPage.create_pages(doc):
	        interpreter.process_page(page)

	return(output_string.getvalue())

                
def convert_title_to_filename(title):
    filename = title.lower()
    filename = filename.replace(' ', '_')
    return filename


def split_to_title_and_pagenum(table_of_contents_entry):
    title_and_pagenum = table_of_contents_entry.strip()
    
    title = None
    pagenum = None
    
    if len(title_and_pagenum) > 0:
        if title_and_pagenum[-1].isdigit():
            i = -2
            while title_and_pagenum[i].isdigit():
                i -= 1

            title = title_and_pagenum[:i].strip()
            pagenum = int(title_and_pagenum[i:].strip())
        
    return title, pagenum

In [ ]:
text = convert_pdf_to_string('Aviva_Tan Keng Tiong_BD Associate.pdf')

In [ ]:
text = text.replace('.','')
text = text.replace('\x0c','')
table_of_contents_raw = text.split('\n')

In [ ]:
encoded = text.encode('utf-8')
table_of_contents_raw

In [ ]:
table_of_contents_raw

In [ ]:
def extract_date(text):
  return text
def extract_job_title(text):
  return text
def extract_job_description(text):
  return text

In [ ]:
import re

def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [ ]:
import re

def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

In [ ]:
import re
extraction=[]
name = table_of_contents_raw[0]
email = extract_email(text)
phone = extract_mobile_number(text)
jobs=[]
for sentence in table_of_contents_raw:
  if (re.match())
   jobs.append({
        'Date': extract_date(sentence),
        'Title':extract_job_title(sentence),
        'Description':extract_job_description(sentence)
        })
extraction.append({
    'Name':name,
    'Email':email,
    'Phone':phone,
    'Jobs':jobs
})
print(sentence)


In [ ]:
extraction

[{'Email': None,
  'Jobs': [{'Date': 'Tan Keng Tiong ',
    'Description': 'Tan Keng Tiong ',
    'Title': 'Tan Keng Tiong '},
   {'Date': 'Block 273B Bishan Street 24 #09-114 Singapore 572273 |  ',
    'Description': 'Block 273B Bishan Street 24 #09-114 Singapore 572273 |  ',
    'Title': 'Block 273B Bishan Street 24 #09-114 Singapore 572273 |  '},
   {'Date': 'Mobile: 65- 8112 5271 | Email: tankengtiong@gmailcom ',
    'Description': 'Mobile: 65- 8112 5271 | Email: tankengtiong@gmailcom ',
    'Title': 'Mobile: 65- 8112 5271 | Email: tankengtiong@gmailcom '},
   {'Date': '', 'Description': '', 'Title': ''},
   {'Date': ' ', 'Description': ' ', 'Title': ' '},
   {'Date': '', 'Description': '', 'Title': ''},
   {'Date': 'Overview ', 'Description': 'Overview ', 'Title': 'Overview '},
   {'Date': '', 'Description': '', 'Title': ''},
   {'Date': 'Currently  as  institutional  sales  support  in  Aviva  Investors,  I’m  part  of  the ',
    'Description': 'Currently  as  institutional  sal

In [ ]:
import re
result = re.search('EXPERIENCE(.*)', text.replace('\n',' ').replace('•',''))
print(result.group(1))
arr=[]
stri=''
extraction=[]
companies=[]
designations=[]

for dataLine in (result.group(1)).split(' '):
  if(dataLine.strip()!=''):
    stri += dataLine + ' '
  else:
    arr.append(stri)
    stri=''
    
for abb in arr:
  brr = len(abb.split(' '))
  if((10>brr>2)and abb[0].isupper()):
    extraction.append(abb)

for idx,item in enumerate(extraction):
  if(idx % 2 == 0):
    companies.append(item)
  else:
    designations.append(item)

    Sept 2015 – Current   ABN AMRO Clearing Bank NV, Relationship Manager              Managing pool of hedge fund, proprietary trading clients   trading multi asset classes      Contributed to 20% increase of revenue for my pool of clients   in FY 2016     Onboarding clients through effective management and   communication to internal stakeholders eg Operations, Risk,  Technical teams  Increasing ROE by efficient allocation of clients’ credit line       Strong understanding of Treasury and SBL services     In-depth knowledge of APAC equities & global  futures/options markets     ETF creation/redemption expertise accumulated through   working with fund managers     Opening up new markets and products offerings to clients    Adhering and performing due diligence checks and annual   compliance reviews for clients     Strong negotiation skills for fees renewal and enforcement of   tighter global regulatory requirements      Preparing client risk management and credit proposals 

In [ ]:
keywords='Representative|Salesperson|trainee|assistant|associate|ambassador|analyst|coordinator|visionary|agent|Specialist|consultant|recruiter|Manager|Advisor|Planner|Administrator|Head|Senior|closer|Director|Vice President|VP|President|executive'

In [ ]:
matched_sent=[]
for sent in arr:
  if(len(re.findall("(?:^|(?<= ))(Representative|Salesperson|trainee|assistant|associate|ambassador|analyst|coordinator|visionary|agent|Specialist|consultant|recruiter|Manager|Advisor|Planner|Administrator|Head|Senior|closer|Director|Vice President|VP|President|executive)(?:(?= )|$)",sent,re.IGNORECASE))>0):
    matched_sent.append(sent)

In [ ]:
dates=extract_date_ranges(dateformat1,text)
if not dates:
  dates = extract_date_ranges(dateformat2,text)
  if not dates:
    dates = extract_date_ranges(dateformat3,text)
    if not dates:
      dates = extract_date_ranges(dateformat4,text)

In [ ]:
text

3rd

In [ ]:
pip install pyresparser

In [ ]:
!python -m spacy download en_core_web_sm
!python -m nltk.downloader words

In [ ]:
from pyresparser import ResumeParser
data = ResumeParser('Aviva_Tan Keng Tiong_BD Associate.pdf').get_extracted_data()

/usr/local/lib/python3.7/dist-packages/spacy/util.py:271: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
data

{'college_name': None,
 'company_names': None,
 'degree': None,
 'designation': None,
 'email': 'tankengtiong@gmail.com',
 'experience': ['December 2014 to Present',
  'Business Development Associate | Aviva Investors | Singapore',
  'Key achievements',
  '•  Awarded  outstanding  employee  award  in  2015  for  demonstrating  Aviva',
  'corporate values',
  '•  Ongoing  support  to  APAC  Head  of  Business  Development  on  strategic',
  'partnership  discussion  with  Japanese  prospects  for  distribution  of  liquid',
  'solutions  (Fixed  Income  and  liquid  alternatives)  and  illiquid  solutions  (ranging',
  'from infrastructure, real estate and private loan investments)',
  '•  Contributed to the product development of Asian Investment Grade Bond Fund',
  'by performing fees and performance comparison among key competitors',
  '•  Conducted  product  trainings  to  local  independent  financial  advisors  for  our',
  'Aviva Investors flagship funds, the Multi-strategy (AIMS